In [ ]:
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from scipy.interpolate import BSpline, make_interp_spline

def HisTogramsReg(image, tissue, bins, ylim, xlim):
    imageRGB = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    
    data = {'red channel': [], 'green channel': [], 'distance': []}
    i = 1
    green_intensity = 0
    red_intensity = 0

    #Take data from channels
    for h in range(imageRGB.shape[0]):
        for w in range(imageRGB.shape[1]):
            if i == h:
                green_mean = green_intensity / imageRGB.shape[1]
                red_mean = red_intensity / imageRGB.shape[1]
                data['red channel'].append(red_mean)
                data['green channel'].append(green_mean)
                data['distance'].append(h - 1)
            
                i = i + 1
                green_intensity = 0
                red_intensity = 0
        
            green_intensity = green_intensity + imageRGB[h,w,1]
            red_intensity = red_intensity + imageRGB[h,w,0]


    #Create dataframe
    df = pd.DataFrame(data)

    #convert distance pixels into distance micrometers
    df['distance'] = df['distance'] * 140 / 800
    
    #plot graph 
    distancedata = np.array(df['distance'])
    reddata = np.array(df['red channel'])
    greendata = np.array(df['green channel'])

    new_x = np.linspace(0,ylim,bins)
    
    red_BSpline = make_interp_spline(distancedata, reddata)
    green_BSpline = make_interp_spline(distancedata, greendata)
    
    red_y = red_BSpline(new_x)
    green_y = green_BSpline(new_x)
    
    plt.figure(figsize = (5, 15))
    plt.plot(np.flip(red_y), new_x, 'r')
    plt.plot(np.flip(green_y), new_x, 'g')
    
    if 'Retinal' in tissue:
        plt.ylim(0, ylim)
        plt.xlim(0, xlim)
        plt.ylabel('{} thickness (micras)'.format(tissue), fontsize = 20, )
        
    elif 'RPE' in tissue:
        plt.ylim(0, ylim)
        plt.xlim(0, xlim)
        plt.ylabel('{} thickness (micras)'.format(tissue), fontsize = 20)
        
    plt.xlabel('Channel intensity (AU)', fontsize = 20)
    
    
    return df, plt.show()